In [1]:
import tensorflow as tf
from LoadData import loadData
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
paths = [
    "Custom 1",
        "Custom 2",
        "EfficientNet",
        "Inception",
        "MLP",
        "MobileNetv3",
        "ResNet50",
        "VGG"]

image_size = {
    "Custom 1": True,
    "Custom 2": False,
    "EfficientNet": False,
    "Inception": True,
    "MLP": False,
    "MobileNetv3": False,
    "ResNet50": False,
    "VGG": False,
}

In [3]:
train, val, test = loadData(batch_size = 128, image_size = (224, 224))
train, val, test2 = loadData(batch_size = 128, image_size = (150, 150))
train = 0
val = 0

Found 27558 files belonging to 2 classes.
Using 19291 files for training.
Found 27558 files belonging to 2 classes.
Using 8267 files for validation.
Found 27558 files belonging to 2 classes.
Using 19291 files for training.
Found 27558 files belonging to 2 classes.
Using 8267 files for validation.


In [4]:
numpy_labels = np.zeros(2763)
numpy_images = np.zeros((2763, 224, 224, 3))
i = 0
for images, labels in test:
    numpy_labels[(i * 128):((i+1) * 128)] = labels.numpy()
    numpy_images[(i * 128):((i+1) * 128)] = images.numpy()
    i += 1
    
numpy_labels2 = np.zeros(2763)
numpy_images2 = np.zeros((2763, 150, 150, 3))
i = 0
for images, labels in test2:
    numpy_labels2[(i * 128):((i+1) * 128)] = labels.numpy()
    numpy_images2[(i * 128):((i+1) * 128)] = images.numpy()
    i += 1

In [5]:
def calc_f1score(precision, recall):
    return (2 * (precision * recall)) / (precision + recall)

def calc_precision(matrix):
    return (matrix[1][1] / (matrix[1][1] + matrix[0][1]))

def calc_recall(matrix):
    return (matrix[1][1] / (matrix[1][1] + matrix[1][0]))

In [6]:
fig, ax = plt.subplots(2, constrained_layout=True, figsize=(10, 10))
fig.suptitle("Graphs")

ax[0].set_title('Receiver Operating Characteristic')
ax[0].plot([0, 1], [0, 1],'r--')
ax[0].set_xlim([0, 1])
ax[0].set_ylim([0, 1])
ax[0].set_ylabel('True Positive Rate')
ax[0].set_xlabel('False Positive Rate')


ax[1].set_title('Precision Recall Curve')
ax[1].plot([0, 1], [1, 0],'r--')
ax[1].set_xlim([0, 1])
ax[1].set_ylim([0, 1])
ax[1].set_ylabel('Recall')
ax[1].set_xlabel('Precision')


for model_name in paths:
    model = tf.keras.models.load_model("Models/" + model_name)
    
    if (image_size[model_name]):
        images = numpy_images2
        labels = numpy_labels2
    else:
        images = numpy_images
        labels = numpy_labels
        
        
    predicted_labels = model.predict(images)
    fpr, tpr, threshold = metrics.roc_curve(labels, predicted_labels)
    roc_auc = metrics.roc_auc_score(labels, predicted_labels)

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]

    predicted_labels_thresholded = predicted_labels > optimal_threshold
    
    # eval_result = model.evaluate(images, labels)
    
    confusion_matrix = metrics.confusion_matrix(labels, predicted_labels_thresholded)
    #     cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["Uninfected", "Infected"])

    #     cm_display.plot()
    #     plt.show()

    precision_array, recall_array, thresholds = metrics.precision_recall_curve(numpy_labels, predicted_labels)
    
    ax[0].plot(fpr, tpr, label = "{} = {}".format( model_name, round(roc_auc, 4)))
    
    ax[1].plot(precision_array, recall_array, label = model_name)
    
    precision = calc_precision(confusion_matrix)
    recall = calc_recall(confusion_matrix)
    f1score = calc_f1score(precision, recall)
    
    print(model_name)
    print("Parameter Count: {}".format(round(model.count_params(), 4)))
    # print("Accuracy: {}".format(round(eval_result[1], 4)))
    print("Precision: {}".format(round(precision, 4)))
    print("Recall: {}".format(round(recall, 4)))
    print("F1-Score: {}".format(round(f1score, 4)))
    print("AUC: {}".format(round(roc_auc, 4)))


ax[0].legend()
ax[1].legend()
plt.show()

22/22 [==============================] - 19s 316ms/step - loss: 0.1803 - acc: 0.9377
ResNet50
Accuracy: 0.9377
22/22 [==============================] - 5s 157ms/step - loss: 0.1707 - acc: 0.9359
VGG
Accuracy: 0.9359
